In [16]:
def combine(command, parameter):
    repeat = {}
    i = 0
    while i < len(command):
        line = ''
        count = 0
        space_start = i-1
        while i < len(command):
            if command[i] == '{':
                count += 1
            else:
                if command[i] == ' ' or command[i] == '\t':
                    line += command[i]
                else:
                    line = ''
                    space_start = i
                count = 0
            i += 1      
            if(count == 2):
                start = i
                break
            
        count = 0
        while i < len(command):
            if command[i] == '}':
                count += 1
            else:
                count = 0
            i += 1                
            if count == 2:
                current = command[start:i-2].strip()
                if not current in repeat.keys() or repeat[current] == 0:
                    repeat[current] = 1
                else:
                    repeat[current] += 1
                    current = current + ' ' + str(repeat[current])
                if current in parameter.keys():
                    value = parameter[current]
                    if len(value) > 0:
                        endn = value[-1] == '\n'
                        value = value.split('\n')
                        if value[-1] == '':
                            value = value[:-1]
                        if len(value) > 1:
                            tv = ''
                            for v in value:
                                tv += line + v + '\n'
                            if not endn:
                                tv = tv[:-1]
                        else:
                            tv = line + value[0] + ('\n' if endn else '')
                    else:
                        tv = ''
                    command = command[:space_start + 1] + tv + command[i:]
                    i = space_start
                else:
                    command = command[:start] + ' ' + current + ' }}' + command[i:]
                break
    return command

_dummy = '''{{ body }}'''

_code = '''{{ command }} {{ head }}:
    {{ body }}'''


_for = combine(_code,{
    'command': 'for',
    'head': '{{ pointer }} in {{ collection }}',
})

_if = combine(_code,{
    'command': 'if',
    'head': '{{ condition }}',
})

_else = combine(_code,{
    'command': 'else',
    'head': '',
})


_def = combine(_code,{
    'command': 'def',
    'head': '{{ name }}({{ parameter }})'
})

_call = '{{ name }}({{ parameter }})'


invoke = combine(_call,{
    'parameter': ''
})

_var = '{{ name }} = {{ value }}\n'

_write = combine(_call,{
    'name':'print',
    'parameter':'{{ text }}'
})

_import = 'import {{ module }}\n'
_from_import = 'from {{ module }} import {{ component }}\n'
_from_import_all = combine(_from_import,{
    'component': '*'
})


function = combine(_def,{
    'name'         : 'bar',
    'parameter'    : 'array',
    'body'         : _for,
    'collection'   : 'array',
    'pointer'      : 'i',
    'body 2'       : _for,
    'collection 2' : 'i',
    'pointer 2'    : 'j',
    'body 3'       : _write,
    'text'         : 'j'
})

print(function)

def bar(array):
    for i in array:
        for j in i:
            print(j)


In [6]:
imports = combine(_dummy,
{
    'body': 
            _from_import_all * 5 +
            _import + '\n' +
            invoke + '\n' + 
            (_call + '\n') * 2 +
            _var +
            _call + '\n',
    'module'  : 'pygame.locals',
    'module 2': 'OpenGL.GL',
    'module 3': 'OpenGL.GLU',
    'module 4': 'OpenGL.GLUT',
    'module 5': 'geo_math',
    'module 6': 'pygame',
    'name': 'pygame.init',
    'name 2': 'pygame.display.set_caption',
    'parameter': '\'PyGame OpenGL\'',
    'name 3': 'pygame.display.set_icon',
    'parameter 2': 'pygame.image.load(\'icon.png\')',
    'name 4': 'screen',
    'value': '(800,600)',
    'name 5': 'pygame.display.set_mode',
    'parameter 3': 'screen, DOUBLEBUF|OPENGL',
})
print(imports)

from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
from geo_math import *
import pygame

pygame.init()
pygame.display.set_caption('PyGame OpenGL')
pygame.display.set_icon(pygame.image.load('icon.png'))
screen = (800,600)
pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)

